## Use Case: Customer Support Automation

*[Coding along with the Udemy online course [Mastering AutoGen: Building Multi-Agent Systems](https://www.udemy.com/course/autogen-agent-systems/?couponCode=MTST7102224A2) by Paulo Dichone]*

- Automating customer support processing using AutoGen multi Agent systems
- Solving issues and excalte cases to human agents when necessary

__Workflow:__

1. Inquiries are coming in from clients/customers
2. These inquiries are taken by the __Customer Inquiry Agent__ who classifies them.
3. And then passed along to the __Response Agent__ who provides automated responses provided by classifications.
4. For the issue resolution the conversation is passed on to the __Knowledege Base Agent__. He has the ability to search the company's knowledge base.
5. Now case gets passed to the __Trouble Shooting Agent__ who knows how to guide the customer through steps of trouble shooting this issue.
6. Next is the __Escalation Agent__ (did we miss the __Feedback Agent__?) who'll identify the cases the require human interaction.
7. In the final step there's the __Human Support Agent__ that, if necessary, would connect the customer with a *human support representative*.

In [1]:
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent
import pandas as pd
from pprint import pprint

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [3]:
llm_config = {
    "model": "gpt-4o-mini",
    "temperature": 0.4,
    "api_key": api_key
    }
print("Don't be a fool and send your api key to GitHub!")

Don't be a fool and send your api key to GitHub!


In [4]:
# define the customer inquiry agent
inquiry_agent = ConversableAgent(
    name="Inquiry_Agent",
    llm_config=llm_config,
    system_message="You handle customer inquiries and classify them.",
)

In [5]:
# define the response agent
response_agent = ConversableAgent(
    name="Response_Agent",
    llm_config=llm_config,
    system_message="You provide automated responses based on the inquiry classification.",
)

In [6]:
# define the knowledge base agent
knowledge_base_agent = ConversableAgent(
    name="Knowledge_Base_Agent",
    llm_config=llm_config,
    system_message="You search the company's knowledge base for solutions to customer issues.",
)

In [7]:
# define the troubleshooting agent
troubleshooting_agent = ConversableAgent(
    name="Troubleshooting_Agent",
    llm_config=llm_config,
    system_message="You guide customers through troubleshooting steps to resolve their issues.",
)

In [8]:
# define the feedback agent
feedback_agent = ConversableAgent(
    name="Feedback_Agent",
    llm_config=llm_config,
    system_message="You collect customer feedback on the resolution process.",
)

In [9]:
# define the escalation agent
escalation_agent = ConversableAgent(
    name="Escalation_Agent",
    llm_config=llm_config,
    system_message="You identify cases that require human intervention.",
)

In [11]:
# define the human support agent
human_support_agent = ConversableAgent(
    name="Human_Support_Agent",
    llm_config=llm_config,
    system_message="You connect customers with human support representatives.",
)

In [12]:
# define the user proxy agent
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "my_code",
        "use_docker": False,
    },
)

In [13]:
# register nested chats with the user proxy agent
user_proxy.register_nested_chats(
    [
        {
            "recipient": response_agent,
            "message": lambda recipient, messages, sender, config: f"Classify and respond to this inquiry: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": knowledge_base_agent,
            "message": lambda recipient, messages, sender, config: f"Search for solutions to this issue: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": troubleshooting_agent,
            "message": lambda recipient, messages, sender, config: f"Guide through troubleshooting for this issue: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": feedback_agent,
            "message": lambda recipient, messages, sender, config: f"Collect feedback on this resolution process: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
        {
            "recipient": escalation_agent,
            "message": lambda recipient, messages, sender, config: f"Determine if this case needs human intervention: {messages[-1]['content']}",
            "summary_method": "last_msg",
            "max_turns": 1,
        },
    ],
    trigger=inquiry_agent,
)

In [14]:
# define the initial customer inquiry
initial_inquiry = (
    """My internet is not working, and I have already tried rebooting the router."""
)

In [15]:
# start the nested chat
user_proxy.initiate_chat(
    recipient=inquiry_agent,
    message=initial_inquiry,
    max_turns=2,
    summary_method="last_msg",
)

User (to Inquiry_Agent):

My internet is not working, and I have already tried rebooting the router.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Inquiry_Agent (to User):

This inquiry can be classified as a **Technical Support Request** related to internet connectivity issues.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
User (to Response_Agent):

Classify and respond to this inquiry: This inquiry can be classified as a **Technical Support Request** related to internet connectivity issues.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Response_Agent (to User):

**Response:** 

Thank you for reaching out regarding your internet connec

ChatResult(chat_id=None, chat_history=[{'content': 'My internet is not working, and I have already tried rebooting the router.', 'role': 'assistant', 'name': 'User'}, {'content': 'This inquiry can be classified as a **Technical Support Request** related to internet connectivity issues.', 'role': 'user', 'name': 'Inquiry_Agent'}, {'content': 'This case does not require human intervention at this stage. The automated response provides a comprehensive set of troubleshooting steps for the user to follow regarding their internet connectivity issues. It encourages engagement by asking for feedback and results after each step, which can help in diagnosing the problem further. However, if the user indicates that the issue persists after following all the suggested steps, then human intervention may be necessary to provide more specialized assistance.', 'role': 'assistant', 'name': 'User'}, {'content': 'This inquiry can be classified as a **Technical Support Request** with a suggestion for an *